# Do Palace Merging

In [ ]:
import importlib
import pickle
from pathlib import Path

import pandas as pd

import gtsfm.products.scene_tree as scene_tree
from gtsfm.products.visibility_graph import (VisibilityGraph,
                                             visibility_graph_keys)
from gtsfm.utils.tree import PostOrderIter

importlib.reload(scene_tree)
import gtsfm.common.gtsfm_data as gtsfm_data

importlib.reload(gtsfm_data)
from gtsfm.products.scene_tree import SceneTree

PALACE = Path("../palace_metis")
TESTS_PALACE = Path("../tests/data/palace")

pseudo_color = True
down_sample = True

In [ ]:
# Just FYI
df = pd.read_csv(TESTS_PALACE / 'visibility_graph.csv')
graph : VisibilityGraph = list(zip(df["i"], df["j"]))
print("Number of edges in visibility graph:", len(graph))
print("Number of keys:", len(visibility_graph_keys(graph)))

In [ ]:
# Needed for re-ordering
with open(TESTS_PALACE / 'cluster_tree.pkl', 'rb') as f:
    cluster_tree = pickle.load(f)
print('Loaded cluster_tree from', TESTS_PALACE / 'cluster_tree.pkl')
cluster_tree

In [ ]:
# Big slow read!
tree = scene_tree.read_colmap(PALACE, "sparse")

In [ ]:
# Show sizes
assert tree is not None
for node in PostOrderIter(tree):
    print(node.value)
    
if down_sample:
    seed = 42
    fraction_points_to_keep = 0.1
    tree = scene_tree.downsample(tree, fraction_points_to_keep=fraction_points_to_keep, seed=seed)

if pseudo_color:
    tree = scene_tree.color_by_cluster(tree)

reordered_tree: SceneTree = scene_tree.reorder(tree, cluster_tree)

# Given a cluster tree with *local* VGGT results, we want to output a single GtsfmData that is
# the merger of all the cluster results.
merged_scene = scene_tree.merge(reordered_tree)
print(merged_scene)

# TODO(frank): these images names and shapes should be real.
image_filenames = [f"San_Francisco_{i:04d}.jpg" for i in range(281)]  # Dummy filenames
image_shapes = [(1936, 1296)] * len(image_filenames)  # Dummy shapes
merged_scene.export_as_colmap_text(
    Path("../tests/data/palace_merged_colmap") / "pseudo-10%",
    image_filenames=image_filenames,
    image_shapes=image_shapes,
)
